In [ ]:
import pandas as pd
from pandarallel import pandarallel
import ast
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
dataRew=pd.read_json('../Dataset/IMDB_reviews.json',lines=True)
dataMovie=pd.read_json('../Dataset/IMDB_movie_details.json',lines=True)

In [ ]:
dataRew.info()

In [ ]:
dataRew[dataRew['review_text']=='']

In [ ]:
dataMovie.info()

In [ ]:
dataMovie[dataMovie["plot_synopsis"]=='']

In [ ]:
dataMovie.loc[2]

In [ ]:
dataMovie.head()

In [ ]:
dataRew.head()

In [ ]:
ColonnaL=dataRew["review_text"].apply(len)

1460 caratteri di media ogni recensione

In [ ]:
ColonnaL.mean()

In [ ]:
dataRew.groupby(dataRew["is_spoiler"]).nunique()

In [ ]:
dataRew.isnull().sum()

In [ ]:
dataMovie.isnull().sum()

In [ ]:
unique_data=dataRew["review_date"].unique()

In [ ]:
data = pd.to_datetime(unique_data)

In [ ]:
unique_data

In [ ]:
data

In [ ]:
data.min()

In [ ]:
data.max()

### Distribution of spoilers

In [ ]:
values=dataRew["is_spoiler"].value_counts()

In [ ]:
values.plot(kind='bar',color=['blue','red'])
plt.xlabel('Spoiler')
plt.ylabel('Numbers reviews')
plt.title('Distribution of Spoiler in reviews')

## Review Cleaning

In [ ]:
nltk.download('punkt')

In [ ]:
pandarallel.initialize(progress_bar=True)

### Tokenization

In [ ]:
def tokenize_text(text):
    from nltk.tokenize import WordPunctTokenizer
    tokenizer=WordPunctTokenizer()
    return tokenizer.tokenize(text)


In [ ]:
dataRew['clean_review'] = dataRew.loc[:,"review_text"].parallel_apply(tokenize_text)

In [ ]:
dataRew.to_csv('../Dataset/datiCleanTokenize.csv', index=False)

### Delete StopWords

In [ ]:
def takeDataset(x):
    ## When i read the csv, the last column is not an array but a string
    dataRew=pd.read_csv(x)
    ## Converting the string in an array before delete stop words
    dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

In [ ]:
nltk.download('stopwords')

In [ ]:
def remove_Stop(x):
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  filtered_sentence = [word for word in x if word.lower() not in stop_words]
  return filtered_sentence

In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(remove_Stop)

In [ ]:
dataRew.to_csv('../Dataset/datiCleanTokenizeStop.csv', index=False)

### Regex to  remove of punct.

In [ ]:
dataRew["clean_review"]=dataRew.loc[:,"clean_review"].parallel_apply(ast.literal_eval)

In [ ]:
def remove_punct2(words_list):
    import re
    pattern=re.compile(r'[^\w\s]')
    clean_words=[word for word in words_list if not pattern.search(word)]
    return clean_words

In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(remove_punct2)

In [ ]:
dataRew["clean_review"]

In [ ]:
dataRew.to_csv('../Dataset/datiCleanTokenizeStopNotPun.csv', index=False)

In [ ]:
dataRew.loc[:,"clean_review"]

### Lemmatization

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
def lemmatize(text):
    from nltk.stem import WordNetLemmatizer
    lemmatizer=WordNetLemmatizer()
    lem_token=[lemmatizer.lemmatize(token.lower()) for token in text]
    return lem_token


In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(lemmatize)

In [ ]:
dataRew.loc[:,"clean_review"]

In [ ]:
dataRew.to_csv('../Dataset/datiClean.csv', index=False)

### Stemming

In [ ]:
def stem_text(x):
  from nltk.stem import SnowballStemmer
  stemmer=SnowballStemmer('english')
  s_words=[stemmer.stem(word) for word in x]
  return s_words

In [ ]:
dataRew['clean_review']=dataRew.loc[:,'clean_review'].parallel_apply(stem_text)

In [ ]:
dataRew.loc[3,"clean_review"]

##  Clean Summary of Movie

### Tokenize

In [ ]:
def tokenize_text(text):
   from nltk.tokenize import WordPunctTokenizer
   tokenizer=WordPunctTokenizer()
   return tokenizer.tokenize(text)

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_synopsis"].parallel_apply(tokenize_text)

### Remove Stop Words

In [ ]:
nltk.download('stopwords')
def remove_Stop(x):
   from nltk.corpus import stopwords
   stop_words=set(stopwords.words('english'))
   filtered_sentence=[word for word in x if word.lower() not in stop_words]
   return filtered_sentence

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(remove_Stop)

In [ ]:
dataMovie["plot_clean"]

### Remove punctuation

In [ ]:
def remove_punct(x):
   import string
   senzaP=[parola for parola in x if parola not in string.punctuation]
   return senzaP

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(remove_punct)

In [ ]:
dataMovie["plot_clean"]

### Stemming

In [ ]:
def Stem(x):
   from nltk.stem import PorterStemmer
   stemmer=PorterStemmer()
   words=[stemmer.stem(word) for word in x]
   return words

In [ ]:
dataMovie["plot_clean"]=dataMovie.loc[:,"plot_clean"].parallel_apply(Stem)

### Remove Punctuaction 2

In [ ]:
dataMovie['plot_clean']=dataMovie.loc[:,'plot_clean'].parallel_apply(remove_punct2)

In [ ]:
dataMovie.to_csv("../Dataset/movieclean.csv")

In [ ]:
dataMovie